# Step 9: Training with LLM

## Latar Belakang

In [1]:
import pandas as pd
from datasets import Dataset

# Muat dataset
data = pd.read_csv('data/final-data/latarbelakang.csv')

data = data[['kalimat', 'summary']]


c:\Users\harry\anaconda3\envs\torch-nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sklearn.model_selection import train_test_split

# Membagi data
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

print(f"Training set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")

Training set size: 8
Validation set size: 2


In [3]:
train_data.to_csv('data/model-data/train.csv', index=False)
val_data.to_csv('data/model-data/val.csv', index=False)

In [4]:
train_data = pd.read_csv('data/model-data/train.csv')
val_data = pd.read_csv('data/model-data/val.csv')

In [5]:
dataset = Dataset.from_pandas(data)
val_dataset = Dataset.from_pandas(val_data)
train_dataset = Dataset.from_pandas(train_data)

In [6]:
import torch_directml

device = torch_directml.device(torch_directml.default_device())

In [7]:
# Load model dan tokenizer
from transformers import EncoderDecoderModel, BertTokenizer

model = EncoderDecoderModel.from_pretrained("cahya/bert2bert-indonesian-summarization").to(device)
tokenizer = BertTokenizer.from_pretrained("cahya/bert2bert-indonesian-summarization")

# Fungsi untuk preprocess dan tokenisasi data
def preprocess_data(examples):
    inputs = tokenizer(examples['kalimat'], truncation=True, padding="max_length", max_length=512, return_tensors="pt")
    labels = tokenizer(examples['summary'], truncation=True, padding="max_length", max_length=256, return_tensors="pt")
    
    # Pindahkan ke DirectML device
    inputs['input_ids'] = inputs['input_ids'].to(device)
    inputs['attention_mask'] = inputs['attention_mask'].to(device)
    labels['input_ids'] = labels['input_ids'].to(device)
    
    inputs['labels'] = labels['input_ids']
    return inputs

# Terapkan preprocess pada dataset
tokenized_dataset = dataset.map(preprocess_data, batched=True)
train_tokenized_dataset = train_dataset.map(preprocess_data, batched=True)
val_tokenized_dataset = val_dataset.map(preprocess_data, batched=True)

c:\Users\harry\anaconda3\envs\torch-nlp\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|██████████| 2/2 [00:00<00:00, 23.83 examples/s]


In [8]:
from transformers import TrainingArguments, Trainer

# Tentukan parameter training
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,  # Sesuaikan batch size dengan dataset kecil
    per_device_eval_batch_size=1,   # Sesuaikan batch size dengan dataset kecil
    evaluation_strategy="epoch",    # Evaluasi setiap epoch
    save_total_limit=1,             # Simpan checkpoint terbaik
    num_train_epochs=5,             # Tambah epoch karena dataset kecil
    logging_steps=1,                # Logging lebih sering karena dataset kecil
    save_steps=1,                   # Simpan lebih sering
    warmup_steps=10,                # Tambah warmup karena dataset kecil
    weight_decay=0.01,
    logging_dir="./logs",
)

c:\Users\harry\anaconda3\envs\torch-nlp\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_dataset,
    eval_dataset=val_tokenized_dataset,
)

trainer.train()

  0%|          | 0/40 [00:00<?, ?it/s]c:\Users\harry\anaconda3\envs\torch-nlp\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:623: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
c:\Users\harry\anaconda3\envs\torch-nlp\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRE

{'loss': 6.6537, 'grad_norm': 96.01997375488281, 'learning_rate': 5e-06, 'epoch': 0.12}


c:\Users\harry\anaconda3\envs\torch-nlp\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:623: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
c:\Users\harry\anaconda3\envs\torch-nlp\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
  5%|▌ 

{'loss': 7.7854, 'grad_norm': 85.61837768554688, 'learning_rate': 1e-05, 'epoch': 0.25}


c:\Users\harry\anaconda3\envs\torch-nlp\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:623: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
c:\Users\harry\anaconda3\envs\torch-nlp\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
  8%|▊ 

...

c:\Users\harry\anaconda3\envs\torch-nlp\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:623: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
c:\Users\harry\anaconda3\envs\torch-nlp\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
100%|██

{'loss': 0.959, 'grad_norm': 9.52935791015625, 'learning_rate': 0.0, 'epoch': 5.0}


c:\Users\harry\anaconda3\envs\torch-nlp\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:623: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
c:\Users\harry\anaconda3\envs\torch-nlp\lib\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
       

{'eval_loss': 3.850471019744873, 'eval_runtime': 2.2296, 'eval_samples_per_second': 0.897, 'eval_steps_per_second': 0.897, 'epoch': 5.0}
{'train_runtime': 488.9056, 'train_samples_per_second': 0.082, 'train_steps_per_second': 0.082, 'train_loss': 3.6377438813447953, 'epoch': 5.0}


TrainOutput(global_step=40, training_loss=3.6377438813447953, metrics={'train_runtime': 488.9056, 'train_samples_per_second': 0.082, 'train_steps_per_second': 0.082, 'total_flos': 27562176675840.0, 'train_loss': 3.6377438813447953, 'epoch': 5.0})

In [29]:
import torch
# Fungsi untuk memilih device
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    else:
        return torch.device("cpu")
    
device = get_device()

In [30]:
def summarize(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    summary_ids = model.generate(inputs['input_ids'], max_length=256, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [31]:
test_text = "kulit adalah organ terbesar dalam tubuh dan memainkan peran penting dalam pertahanan terhadap ancaman lingkungan seperti bakteri virus dan zat berbahaya. penyakit kulit merupakan masalah kesehatan yang tersebar luas dan menyerang orang orang dari segala usia dan penyakit ini dapat disebabkan oleh berbagai faktor seperti genetika gaya hidup dan paparan lingkungan. kelainan kulit merupakan hal y ang umum terjadi di seluruh dunia termasuk di indonesia dimana penyakit ini menimbulkan tantangan besar terhadap kesehatan masyarakat karena tingginya angka kejadiannya j. zhang et al. 2023 . indonesia adalah rumah bagi lebih dari 270 juta orang yang terkena berbagai penyakit kulit menjadikannya salah satu negara yang paling terkena dampaknya secara global. penyakit kulit yang umum termasuk psoriasis eksim acne vulgaris vitiligo dan melano ma. penyakit penyakit ini tidak hanya menimbulkan gejala fisik tetapi juga mempunyai dampak psikologis yang mendalam mempengaruhi kualitas hidup pasien dan interaksi sosial. diagnosis yang tepat dan efisien dari penyakit kulit menjadi kunci dalam penangan an yang efektif. namun proses diagnosis yang tergantung pada pengalaman klinis dan pengetahuan medis yang mendalam dari dermatolog sering kali membatasi akses terhadap perawatan yang berkualitas. diagnosis penyakit kulit secara tradisional memerlukan pemeriksaan langsung oleh seorang dokter spesialis kulit yang kemudian dianalisis berdasarkan pengalaman klinis dan pengetahuan medis yang dimilikinya. dengan perkembangan teknologi terutama di bidang kecerdasan buatan artificial intelligence atau ai ada potensi untuk mengoptimalkan proses diagnosis penyakit kulit. dalam beberapa tahun terakhir kemajuan dalam bidang kecerdasan buatan telah membuka peluang baru dalam peningkatan diagnosis penyakit kulit. pemanfaatan algoritma kecerdasan buatan telah terbukti efektif dalam mendukung identifikasi penyakit kulit yang dapat meningkatkan akurasi dan kecepatan diagnosis. namun tantangan uta ma yang masih dihadapi adalah integrasi teknologi ini ke dalam praktik klinis sehari hari dengan cara yang efisien dan aman. penelitian sebelumnya telah menyoroti potensi pengembangan metode identifikasi pra diagnosis penyakit kulit melalui pemanfaatan algoritma kecerdasan buatan. pengenalan metode ini diharapkan dapat mengoptimalkan aplikasi medical dermatology dengan memperken alkan pendekatan yang sistematis dan terstruktur dalam pengenalan gejala penyakit kulit. pada penelitian wei et al. 2018 membahas tentang pengenalan penyakit kulit berbasis image color dan fitur tekstur. metode yang digunakan meliputi glcm untuk analisis tekstur svm untuk klasifikasi. hasil akhir fitur warna dan tekstur yang digabungkan meningkatkan akurasi pengenalan hing ga lebih dari 90. pada penelitian fauzi dzulfiqar wibowo et.al 2022 membahas tentang deteksi penyakit kulit di indonesia menggunakan cnn dan pembelajaran ensemble . dataset mencakup 1203 gambar penyakit kulit untuk klasifikasi. metode pembelajaran cnn dan ensemble untuk deteksi gambar penyakit kulit. validasi silang k fold untuk pemisahan data dan hasil yang optimal . prapemrosesan data dengan arsitektur cnn resnet50 untuk pelatihan. hasil akhir adalah gambar rgb memiliki akurasi 49 gambar grayscale memiliki akurasi 47 . pada penelitian raghav agarwal et.al 2023 membahas tentang artificial intelligence untuk deteksi gangguan kulit menggunakan model deep learning convolutional neural network. dataset mencakup 25000 gambar warna dari delapan gangguan kulit umum. metode yang digunakan merupakan resnet 152 turunan cnn dalam pemrosesan klasifikasi citra. hasil akhir yang didapat pada penelitian ini dengan menggunakan resnet152 adalah 74.24 pada validation and 73.01 pada test dataset . pada penelitian yue shen et.al 2024 membahas tentang m engoptimalkan diagnosis penyakit kulit memanfaatkan data komunitas daring dengan teknik ontrastive learning dan clustering . metode pendekatan contrastive learning untuk mempelajari representasi umum dari gambar yang tidak berlabel. menggunakan metode clustering dengan gambar validasi standar untuk meningkatkan anotasi. hasil akhir adalah m odel dapat ditransfer ke tugas baru dengan akurasi 6176. dan pada penelitian archana et.al 2019 membahas klasifikasi m odel deteksi penyakit kulit menggunakan pemrosesan gambar untuk aksesibilitas pedesaan . transformasi dct dwt dan svd digunakan untuk deteksi penyakit kulit . hasil akhir akurasi adalah 80 persen. pada penelitian ini mengusulkan pendekatan baru terhadap pra diagnosis penyakit kulit manusia di indonesia melalui pengembangan metode identifikasi berbasis ai yang memanfaatkan data klinis dan non klinis. dengan mengintegrasikan data dari berbagai sumber termasuk rekam medis pasien informasi genetik paparan lingkungan dan kebiasaan gaya hidup metode yang diusulkan ini bertujuan untuk meningkatkan akurasi dan kecepatan diagnosis penyakit kulit sehingga memberikan hasil yang lebih baik bagi pasien. met ode yang diusulkan juga mempertimbangkan karakteristik unik penduduk indonesia termasuk variasi penyakit kulit dan preferensi pasien untuk memastikan relevansi dan efektivitasnya dalam konteks ini. pendekatan ini merupakan perubahan signifikan dari metod e diagnostik tradisional dan berpotensi merevolusi bidang dermatologi medis di indonesia dan berbagai negara lain. dengan demikian penelitian ini bertujuan untuk mengisi kesenjangan pengetahuan dengan mengembangkan pendekatan ilmiah yang lebih sistematis dalam pra diagnosis penyakit kulit menggunakan algoritma kecerdasan buatan. melalui investigasi yang komprehensif diharapkan dapat dihasilkan metode identifikasi yang efisien dan akurat yang mampu mendukung praktik klinis dan nonklinis dalam menghadapi berbagai tantangan dalam diagnosis penyakit kulit. pengembangan metode identifikasi ini diharapkan dapat meningkatkan aksesibilitas terhadap perawatan kesehatan kulit mengurangi waktu tunggu untuk konsultasi serta memberikan kontribusi signifikan dalam peningkatan kualitas perawatan pasien. selain itu p enelitian ini juga diharapkan dapat memberikan landasan bagi pengembangan teknologi medis yang lebih lanjut dalam bidang dermatologi serta memberikan dampak positif dalam perluasan cakupan layanan kesehatan kulit secara global."
print(summarize(test_text))

penyakit kulit merupakan masalah kesehatan yang tersebar luas dan menyerang orang dari segala usia dan penyakit ini dapat disebabkan oleh berbagai faktor seperti genetika gaya hidup dan paparan lingkungan.


## Rumusan Masalah

## Tujuan Penelitian

## Rangkuman Penelitian Terkait

## Metodologi Penelitian